In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
tqdm.pandas()

c:\Users\thatt\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thatt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data = pd.read_csv(
    "C:/Users/thatt/OneDrive/Documents/Coding Project/MyPyhton/Class/Machine Learning Camp/Project/nlp/information_retreival/data/articles.csv"
)
data = data.drop(list(data.columns)[2:], axis=1)
data.head()

,title,author_id
0,5 Best Practices For Writing SQL Joins,priyankab14
1,Foundation CSS Dropdown Menu,ishankhandelwals
2,Top 20 Excel Shortcuts That You Need To Know,priyankab14
3,Servlet – Fetching Result,nishatiwari1719
4,Suffix Sum Array,rohit768


In [6]:
all_words: list[str] = []

for i in range(len(data)):
    words: list[str] = data["title"][i].split(" ")
    for word in words:
        all_words.append(word.lower())

all_words = list(set(all_words))
# remove stopwords
all_words = [word for word in all_words if word not in stopwords.words("english")]
print("containing", len(all_words), "unique words")

containing 17976 unique words


In [7]:
def bow_encoder(text: str, word_list: list[str] = all_words) -> np.array:
    bow = np.zeros(len(word_list))
    for word in text.split():
        if word in word_list:
            bow[word_list.index(word)] += 1
    return bow

In [8]:
data["embbeding"] = data["title"].progress_apply(lambda x: bow_encoder(x, all_words))

100%|██████████| 34574/34574 [01:04<00:00, 533.29it/s]


In [9]:
def cosine_similarity(a: np.array, b: np.array) -> float:
    return np.dot(a, b) / (np.dot(a, a) ** 0.5) * (np.dot(b, b) ** 0.5)

In [11]:
def get_similar_articles(
    text: int, data: pd.DataFrame = data, n: int = 5, encoder=bow_encoder
) -> pd.DataFrame:
    article_embbeding = encoder(text)
    print("Perform Searching...")
    data["similarity"] = data["embbeding"].progress_apply(
        lambda x: cosine_similarity(article_embbeding, x)
    )
    return (
        data.sort_values("similarity", ascending=False).head(n).reset_index(drop=True)
    )

In [14]:
text = "for loop in python"
results = get_similar_articles(text)
for idx, result in enumerate(results["title"]):
    print(
        str(idx + 1) + ".",
        result,
        "[similarity]:",
        round(results["similarity"][idx], 3),
    )

Perform Searching...


100%|██████████| 34574/34574 [00:08<00:00, 3937.32it/s]

1. Difference between fundamental data types and derived data types [similarity]: 4.472
2. Primitive data type vs. Object data type in Java with Examples [similarity]: 4.243
3. Difference between data type and data structure [similarity]: 3.464
4. Difference between Traditional data and Big data [similarity]: 2.828
5. Python | Plotting charts in excel sheet with data tools using XlsxWriter module | Set – 2 [similarity]: 2.55
